In [55]:
import newspaper
from newspaper import Article
from newspaper import news_pool
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import pickle
import datetime
import gzip 
import advertools as adv
from IPython.display import clear_output 

## Crawling article links from Yahoo Finance News sitemap

In [56]:
exclusion_list = ['/hr-education/', '/leisure-entertainment/', '/media-marketing/', '/professional-services/', '/professional-serviceslegal/']
inclusion_list = ['/news/', '/more-news/']

In [18]:
i = 0
link_list = []
for page in [1, 2]:
    search_url = "https://sbr.com.sg/sitemap.xml?page={}".format(page)
    try:
        sbr_sitemap = adv.sitemap_to_df(search_url)
        page_link_list_raw = sbr_sitemap['loc'].tolist()
        # page_link_list = [link for link in page_link_list_raw if any(
        #     exclude in link for exclude in exclusion_list)]
        link_list.extend(page_link_list_raw)
    except:
        continue


2022-11-01 16:11:16,585 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://sbr.com.sg/sitemap.xml?page=1
2022-11-01 16:11:17,816 | INFO | sitemaps.py:536 | sitemap_to_df | Getting https://sbr.com.sg/sitemap.xml?page=2


In [19]:
len(link_list)

67061

In [20]:
# Save as pickle
with open('sbr_link_list_dump.pkl', 'wb') as f:
    pickle.dump(link_list, f)

Filtering to include only news as identified with '/news/' and '/more-news/' excluding link from these industries - '/hr-education/', '/leisure-entertainment/', '/media-marketing/', '/professional-services/', '/professional-serviceslegal/'

In [57]:
with open('sbr_link_list_dump.pkl', 'rb') as f:
    link_list_dump = pickle.load(f)

In [21]:
len(link_list)

67061

In [58]:
link_list_news = [link for link in link_list_dump if any(
    include in link for include in inclusion_list)]

link_list_news = [link for link in link_list_news if not any(
    excluded in link for excluded in exclusion_list)]

link_list_news = list(set(link_list_news))

In [59]:
len(link_list_news)

42401

In [60]:
# Save as pickle
with open('sbr_link_list_news_more_news.pkl', 'wb') as f:
    pickle.dump(link_list_news, f)

In [61]:
with open('sbr_link_list_news_more_news.pkl', 'rb') as f:
    link_list_news = pickle.load(f)

In [62]:
len(link_list_news)

42401

Set Indexes to crawl from 

In [149]:
# start, end = 0, 5000

# Get Article objects

In [63]:
link_list_news_15k = random.sample(link_list_news, 15000)
len(link_list_news_15k)

15000

In [64]:
# Save as pickle
with open('sbr_link_list_news_more_news_random_15k.pkl', 'wb') as f:
    pickle.dump(link_list_news_15k, f)

In [38]:
from IPython.display import clear_output
import time
results = {}
i = 0
for link in link_list_news_15k:
    try:
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        results[link] = article
        clear_output(wait=True)
        i += 1
        print(i)
    except:
        clear_output(wait=True)
        print('F')
        continue


50


In [39]:
d = {
    'link': [article for article in results],
    'title': [results[article].title for article in results],
    'summary': [results[article].summary for article in results],
    'text': [results[article].text for article in results],
    # 'top_img': [results[article].top_img for article in results],
    'publish_date': [results[article].publish_date for article in results],
    'keywords': [results[article].keywords for article in results],
    'tags': [results[article].tags for article in results]
}


# Save as JSON

In [40]:
df = pd.DataFrame(data=d)
df['publish_date'] = pd.to_datetime(df['publish_date'], utc=True)
df['publish_date'] = df['publish_date'].dt.strftime('%Y/%m/%d, %H:%M:%S')
result = df.to_json(orient="records")
with open(f'sbr_news_data_15k.json', 'w') as outfile:
    outfile.write(result)


### Open JSON File


In [40]:
import json 
f = open('sbr_news_data_15k.json')
data = json.load(f)
len(data)

50

In [10]:
a = Article("https://sbr.com.sg/energy-offshore/news/keppels-profit-down-221-1603m-in-q2")
a.download()
a.parse()
a.nlp()
a.text

'Its O&M division also performed poorly and lost $60m.\n\nKeppel Corporation struggled to keep their records up as their profits for the second quarter reached only $160.3m, a 22.1% drop from last year\'s.\n\nAccording to the group’s quarterly report, its Offshore and Marine (O&M) division had a net profit of $1.4m, far from last year\'s $61.4m, no thanks to cost-cutting and downsizing.\n\nIn a statement, Keppel Chief Executive Officer Loh Chin Hua said, "Rightsizing efforts undertaken by Keppel O&M helped to achieve an operating profit of $36m, but it was not sufficient to cover higher financing costs for the enlarged working capital."\n\nMeanwhile, OCBC Investment Research reported that the group’s profits from its other divisions continued to rise steadily. Keppel gained $96.6m profit from properties and $24.6m from infrastructure.\n\nOCBC added that the company retained its dividend of 8 S-cents per share, which was similar to last year\'s amount.\n\n'

In [42]:
data[0]

{'link': 'https://sbr.com.sg/energy-offshore/news/keppels-profit-down-221-1603m-in-q2',
 'title': "Keppel's profit down 22.1% to $160.3m in Q2",
 'summary': "Its O&M division also performed poorly and lost $60m.\nKeppel Corporation struggled to keep their records up as their profits for the second quarter reached only $160.3m, a 22.1% drop from last year's.\nAccording to the group’s quarterly report, its Offshore and Marine (O&M) division had a net profit of $1.4m, far from last year's $61.4m, no thanks to cost-cutting and downsizing.\nMeanwhile, OCBC Investment Research reported that the group’s profits from its other divisions continued to rise steadily.\nOCBC added that the company retained its dividend of 8 S-cents per share, which was similar to last year's amount.",
 'text': 'Its O&M division also performed poorly and lost $60m.\n\nKeppel Corporation struggled to keep their records up as their profits for the second quarter reached only $160.3m, a 22.1% drop from last year\'s.\n\

In [53]:
from textblob import TextBlob
for i in range(10):
    text_blob = TextBlob(data[i]['title'])
    text_sentiment = text_blob.sentiment_assessments
    print(text_sentiment.polarity)
    text_sentiment.polarity

-0.15555555555555559
0.0
0.0
-0.1134920634920635
0.0
-0.2
0.0
0.0
0.0
-0.3


In [54]:
data[0:10]

[{'link': 'https://sbr.com.sg/energy-offshore/news/keppels-profit-down-221-1603m-in-q2',
  'title': "Keppel's profit down 22.1% to $160.3m in Q2",
  'summary': "Its O&M division also performed poorly and lost $60m.\nKeppel Corporation struggled to keep their records up as their profits for the second quarter reached only $160.3m, a 22.1% drop from last year's.\nAccording to the group’s quarterly report, its Offshore and Marine (O&M) division had a net profit of $1.4m, far from last year's $61.4m, no thanks to cost-cutting and downsizing.\nMeanwhile, OCBC Investment Research reported that the group’s profits from its other divisions continued to rise steadily.\nOCBC added that the company retained its dividend of 8 S-cents per share, which was similar to last year's amount.",
  'text': 'Its O&M division also performed poorly and lost $60m.\n\nKeppel Corporation struggled to keep their records up as their profits for the second quarter reached only $160.3m, a 22.1% drop from last year\'s

In [34]:
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...


In [52]:
test_text = 'This was the worst movie EVER MADE!!!'
vader = SentimentIntensityAnalyzer()
for i in range(10):
    polarity_scores = vader.polarity_scores(data[i]['title'])
    print(polarity_scores)



{'neg': 0.0, 'neu': 0.707, 'pos': 0.293, 'compound': 0.4404}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.161, 'neu': 0.593, 'pos': 0.246, 'compound': 0.25}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'compound': 0.3182}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.412, 'neu': 0.462, 'pos': 0.126, 'compound': -0.8137}
